# Importar las librerías necesarias
Salvo funciones muy sencillas, la mayoría de las funciones de Python están contenidas en librerías, las cuales requieren ser importadas. Para ello podemos usar el comando `import`. La palabra clave `as` nos permite darle un nombre distinto a la librería que importamos. También, usando `from` podemos importar las funciones que queremos de una librería dada.

En este caso importamos `eigvals`, para calcular los autovalores de una matriz, y también las librerías `numpy` (para cálculo numérico, contiene varias funciones matemáticas).

In [3]:
import numpy as np
from numpy.linalg import eigvals

# Elementos de matriz del operador posición
Los elementos de matriz de los operadores posición y momento están dados por las siguientes fórmulas:
$$\left < m|\hat{x}|n \right > = \sqrt{\frac{n}{2}} \delta_{m, n-1} + \sqrt{\frac{n+1}{2}} \delta_{m, n+1},$$
$$\left < m|\hat{p}|n \right > = -i \sqrt{\frac{n}{2}} \delta_{m, n-1} + i \sqrt{\frac{n+1}{2}} \delta_{m, n+1}.$$

Consecuentemente, definimos las siguientes funciones en Python para obtenerlos:

In [ ]:
def x_elem(m,n):
    if m == n - 1:
        return np.sqrt(n/2.)
    elif m == n + 1:
        return np.sqrt((n+1)/2.)
    else:
        return 0

def p_elem(m,n):
    if m == n - 1:
        return -1j*np.sqrt(n/2.)
    elif m == n + 1:
        return 1j*np.sqrt((n+1)/2.)
    else:
        return 0

A continuación definimos funciones para crear la representación matricial de los operadores. Esto es, no solo obtener un elemento de matriz, sino todos los elementos en una base de `N` autofunciones del oscilador armónico.

In [ ]:
# Matrix representation of the position operator
def X(N):
    mat = np.zeros([N, N])
    for m in range(N):
        for n in range(N):
            mat[m, n] = x_elem(m, n)

    return mat

# Matrix representation of the momentum operator
def P(N):
    mat = np.zeros([N, N], dtype=complex)
    for m in range(N):
        for n in range(N):
            mat[m, n] = p_elem(m, n)

    return mat

¿Tiene sentido lo que estamos haciendo? Reconstruyamos la representación matricial del Hamiltoniano original:
$$\hat{H} = \hat{p}^2 + \hat{x}^2$$
y mostrémosla en pantalla.

In [ ]:
matP2 = P(5).dot(P(5))
matX2 = X(5).dot(X(5))

print(matP2 + matX2)